In [1]:
import autograd.numpy as np
from autograd.scipy.special import logsumexp,erf
from autograd.scipy.stats import norm
from autograd import grad

import os
#from autograd_gamma import gamma, gammainc, gammaincc, gammaincln, gammainccln

import matplotlib.pyplot as plt
from copy import deepcopy
from itertools import product
import pickle

from tree import Tree
from optimizer import Optimizer

import warnings
warnings.filterwarnings('ignore')

np.random.seed(0)

/Users/evsi8432/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
ds = "DS14"

# load in data from fasta
data = {}
filename = "../dat/"+ds+"/"+ds+".fasta"
with open(filename, 'r') as f:
    for line in f:
        if line.startswith(">"):
            species = line.replace(">","").replace("\n","")
            data[species] = ""
        else:
            data[species] += line.replace(">","").replace("\n","").replace(".","-")


n_tax = len(data)
n_char = len(data[species])

In [3]:
taxa = list(data.keys())
sub_data = {}
for sub_size in [48,24,12,6,3]:
    taxa = np.random.choice(taxa,size=sub_size,replace=False)
    sub_data[sub_size] = {taxon: data[taxon] for taxon in taxa}

In [4]:
# write to pickle file
for N in [48,24,12,6,3]:
    if not os.path.exists("../dat/DS14_" + str(N)):
        os.makedirs("../dat/DS14_" + str(N))
    filename = "../dat/DS14_" + str(N) +"/DS14_" + str(N) + ".pickle"
    with open(filename, 'wb') as handle:
        pickle.dump(sub_data[N], handle)

In [5]:
N = 3/2

In [31]:
N = int(N*2)
ds = "DS14_%d"%N
data = sub_data[N]
n_tax = N
print(N)

KeyError: 96

In [27]:
# write nexus file
filename = "../dat/"+ds+"/"+ds+".nexus"
f = open(filename, "w")
f.write("""#NEXUS

BEGIN DATA;
\tDIMENSIONS NTAX=%d NCHAR=%d;
\tFORMAT MISSING=? GAP=- DATATYPE=DNA;
\tMATRIX
""" % (n_tax,n_char))
for key in data:
    f.write("\t")
    f.write(key.replace("-","_"))
    f.write("\t")
    f.write(data[key])
    f.write("\n")

f.write("""\t;
END;""")
f.close()

In [28]:
# make xml file for BEAST candidate runs

filename = "../dat/"+ds+"/"+ds+".xml"
f = open(filename, "w")
f.write('''<?xml version="1.0" standalone="yes"?>

<!-- Generated by BEAUTi v1.10.4 Prerelease #bc6cbd9                         -->
<!--       by Alexei J. Drummond, Andrew Rambaut and Marc A. Suchard         -->
<!--       Department of Computer Science, University of Auckland and        -->
<!--       Institute of Evolutionary Biology, University of Edinburgh        -->
<!--       David Geffen School of Medicine, University of California, Los Angeles-->
<!--       http://beast.community/                                           -->
<beast version="1.10.4">
	

	<!-- The list of taxa to be analysed (can also include dates/ages).          -->
	<!-- ntax=%d                                                                 -->
	<taxa id="taxa">
''' % n_tax)

for species in data:
    f.write('\t\t<taxon id="%s"/> \n' % species)

f.write('''\t</taxa>
	
	

	<!-- The sequence alignment (each sequence refers to a taxon above).         -->
	<!-- ntax=%d nchar=%d                                                      -->
	<alignment id="alignment" dataType="nucleotide">
''' % (n_tax,n_char))

for species in data:
    f.write("\t\t<sequence>\n")
    f.write('\t\t\t<taxon idref="%s"/>\n'%species)
    f.write('\t\t\t%s\n'%data[species])
    f.write("\t\t</sequence>\n")
f.write("\t</alignment>\n")

f.write("""

	<!-- The unique patterns from 1 to end                                       -->
	<!-- npatterns=933                                                           -->
	<patterns id="patterns" from="1" strip="false">
		<alignment idref="alignment"/>
	</patterns>
	

	<!-- A prior assumption that the population size has remained constant       -->
	<!-- throughout the time spanned by the genealogy.                           -->
	<constantSize id="constant" units="years">
		<populationSize>
			<parameter id="constant.popSize" value="5.0"/>
		</populationSize>
	</constantSize>
	

	<!-- Generate a random starting tree under the coalescent process            -->
	<coalescentSimulator id="startingTree">
		<taxa idref="taxa"/>
		<constantSize idref="constant"/>
	</coalescentSimulator>
	

	<!-- Generate a tree model                                                   -->
	<treeModel id="treeModel">
		<coalescentTree idref="startingTree"/>
		<rootHeight>
			<parameter id="treeModel.rootHeight"/>
		</rootHeight>
		<nodeHeights internalNodes="true">
			<parameter id="treeModel.internalNodeHeights"/>
		</nodeHeights>
		<nodeHeights internalNodes="true" rootNode="true">
			<parameter id="treeModel.allInternalNodeHeights"/>
		</nodeHeights>
	</treeModel>

	<!-- Statistic for sum of the branch lengths of the tree (tree length)       -->
	<treeLengthStatistic id="treeLength">
		<treeModel idref="treeModel"/>
	</treeLengthStatistic>
	

	<!-- Generate a coalescent likelihood                                        -->
	<coalescentLikelihood id="coalescent">
		<model>
			<constantSize idref="constant"/>
		</model>
		<populationTree>
			<treeModel idref="treeModel"/>
		</populationTree>
	</coalescentLikelihood>
	

	<!-- The strict clock (Uniform rates across branches)                        -->
	<strictClockBranchRates id="branchRates">
		<rate>
			<parameter id="clock.rate" value="1.0"/>
		</rate>
	</strictClockBranchRates>
	
	<rateStatistic id="meanRate" name="meanRate" mode="mean" internal="true" external="true">
		<treeModel idref="treeModel"/>
		<strictClockBranchRates idref="branchRates"/>
	</rateStatistic>
	

	<!-- The JC substitution model (Jukes & Cantor, 1969)                        -->
	<HKYModel id="jc">
		<frequencies>
			<frequencyModel dataType="nucleotide">
				<frequencies>
					<parameter id="frequencies" value="0.25 0.25 0.25 0.25"/>
				</frequencies>
			</frequencyModel>
		</frequencies>
		<kappa>
			<parameter value="1.0"/>
		</kappa>
	</HKYModel>

	<!-- site model                                                              -->
	<siteModel id="siteModel">
		<substitutionModel>
			<HKYModel idref="jc"/>
		</substitutionModel>
	</siteModel>

	<!--                                                                         -->
	<statistic id="mu" name="mu">
		<siteModel idref="siteModel"/>
	</statistic>
	

	<!-- Likelihood for tree given sequence data                                 -->
	<treeDataLikelihood id="treeLikelihood" useAmbiguities="false">
		<partition>
			<patterns idref="patterns"/>
			<siteModel idref="siteModel"/>
		</partition>
		<treeModel idref="treeModel"/>
		<strictClockBranchRates idref="branchRates"/>
	</treeDataLikelihood>
	

	<!-- Define operators                                                        -->
	<operators id="operators" optimizationSchedule="default">
		<scaleOperator scaleFactor="0.75" scaleAll="true" ignoreBounds="true" weight="3">
			<parameter idref="treeModel.allInternalNodeHeights"/>
		</scaleOperator>
		<subtreeSlide size="1.0" gaussian="true" weight="30">
			<treeModel idref="treeModel"/>
		</subtreeSlide>
		<narrowExchange weight="30">
			<treeModel idref="treeModel"/>
		</narrowExchange>
		<wideExchange weight="3">
			<treeModel idref="treeModel"/>
		</wideExchange>
		<wilsonBalding weight="3">
			<treeModel idref="treeModel"/>
		</wilsonBalding>
		<scaleOperator scaleFactor="0.75" weight="3">
			<parameter idref="treeModel.rootHeight"/>
		</scaleOperator>
		<uniformOperator weight="30">
			<parameter idref="treeModel.internalNodeHeights"/>
		</uniformOperator>
	</operators>
	

	<!-- Define MCMC                                                             -->
	<mcmc id="mcmc" chainLength="1000000" autoOptimize="true" operatorAnalysis="%s.ops">
		<joint id="joint">
			<prior id="prior">
				<coalescentLikelihood idref="coalescent"/>
				
				
				<strictClockBranchRates idref="branchRates"/>
			</prior>
			<likelihood id="likelihood">
				<treeDataLikelihood idref="treeLikelihood"/>
			</likelihood>
		</joint>
		<operators idref="operators"/>

		<!-- write log to screen                                                     -->
		<log id="screenLog" logEvery="1000">
			<column label="Joint" dp="4" width="12">
				<joint idref="joint"/>
			</column>
			<column label="Prior" dp="4" width="12">
				<prior idref="prior"/>
			</column>
			<column label="Likelihood" dp="4" width="12">
				<likelihood idref="likelihood"/>
			</column>
			<column label="rootHeight" sf="6" width="12">
				<parameter idref="treeModel.rootHeight"/>
			</column>
		</log>

		<!-- write log to file                                                       -->
		<log id="fileLog" logEvery="1000" fileName="%s.log" overwrite="false">
			<joint idref="joint"/>
			<prior idref="prior"/>
			<likelihood idref="likelihood"/>
			<parameter idref="treeModel.rootHeight"/>
			<treeLengthStatistic idref="treeLength"/>
			<parameter idref="constant.popSize"/>
			<parameter idref="clock.rate"/>
			<rateStatistic idref="meanRate"/>
			<treeDataLikelihood idref="treeLikelihood"/>
			<strictClockBranchRates idref="branchRates"/>
			<coalescentLikelihood idref="coalescent"/>
			
		</log>

		<!-- write tree log to file                                                  -->
		<logTree id="treeFileLog" logEvery="1000" nexusFormat="true" fileName="%s.trees" sortTranslationTable="true">
			<treeModel idref="treeModel"/>
			<trait name="rate" tag="rate">
				<strictClockBranchRates idref="branchRates"/>
			</trait>
			<joint idref="joint"/>
		</logTree>
	</mcmc>
	
	<report>
		<property name="timer">
			<mcmc idref="mcmc"/>
		</property>
	</report>
	
</beast>
""" % (ds,ds,ds))

f.close()

In [29]:
# make xml file for BEAST golden run

filename = "../dat/"+ds+"/"+ds+"_golden.xml"
f = open(filename, "w")
f.write('''<?xml version="1.0" standalone="yes"?>

<!-- Generated by BEAUTi v1.10.4 Prerelease #bc6cbd9                         -->
<!--       by Alexei J. Drummond, Andrew Rambaut and Marc A. Suchard         -->
<!--       Department of Computer Science, University of Auckland and        -->
<!--       Institute of Evolutionary Biology, University of Edinburgh        -->
<!--       David Geffen School of Medicine, University of California, Los Angeles-->
<!--       http://beast.community/                                           -->
<beast version="1.10.4">
	

	<!-- The list of taxa to be analysed (can also include dates/ages).          -->
	<!-- ntax=%d                                                                 -->
	<taxa id="taxa">
''' % n_tax)

for species in data:
    f.write('\t\t<taxon id="%s"/> \n' % species)

f.write('''\t</taxa>
	
	

	<!-- The sequence alignment (each sequence refers to a taxon above).         -->
	<!-- ntax=%d nchar=%d                                                      -->
	<alignment id="alignment" dataType="nucleotide">
''' % (n_tax,n_char))

for species in data:
    f.write("\t\t<sequence>\n")
    f.write('\t\t\t<taxon idref="%s"/>\n'%species)
    f.write('\t\t\t%s\n'%data[species])
    f.write("\t\t</sequence>\n")
f.write("\t</alignment>\n")

f.write("""

	<!-- The unique patterns from 1 to end                                       -->
	<!-- npatterns=933                                                           -->
	<patterns id="patterns" from="1" strip="false">
		<alignment idref="alignment"/>
	</patterns>
	

	<!-- A prior assumption that the population size has remained constant       -->
	<!-- throughout the time spanned by the genealogy.                           -->
	<constantSize id="constant" units="years">
		<populationSize>
			<parameter id="constant.popSize" value="5.0"/>
		</populationSize>
	</constantSize>
	

	<!-- Generate a random starting tree under the coalescent process            -->
	<coalescentSimulator id="startingTree">
		<taxa idref="taxa"/>
		<constantSize idref="constant"/>
	</coalescentSimulator>
	

	<!-- Generate a tree model                                                   -->
	<treeModel id="treeModel">
		<coalescentTree idref="startingTree"/>
		<rootHeight>
			<parameter id="treeModel.rootHeight"/>
		</rootHeight>
		<nodeHeights internalNodes="true">
			<parameter id="treeModel.internalNodeHeights"/>
		</nodeHeights>
		<nodeHeights internalNodes="true" rootNode="true">
			<parameter id="treeModel.allInternalNodeHeights"/>
		</nodeHeights>
	</treeModel>

	<!-- Statistic for sum of the branch lengths of the tree (tree length)       -->
	<treeLengthStatistic id="treeLength">
		<treeModel idref="treeModel"/>
	</treeLengthStatistic>
	

	<!-- Generate a coalescent likelihood                                        -->
	<coalescentLikelihood id="coalescent">
		<model>
			<constantSize idref="constant"/>
		</model>
		<populationTree>
			<treeModel idref="treeModel"/>
		</populationTree>
	</coalescentLikelihood>
	

	<!-- The strict clock (Uniform rates across branches)                        -->
	<strictClockBranchRates id="branchRates">
		<rate>
			<parameter id="clock.rate" value="1.0"/>
		</rate>
	</strictClockBranchRates>
	
	<rateStatistic id="meanRate" name="meanRate" mode="mean" internal="true" external="true">
		<treeModel idref="treeModel"/>
		<strictClockBranchRates idref="branchRates"/>
	</rateStatistic>
	

	<!-- The JC substitution model (Jukes & Cantor, 1969)                        -->
	<HKYModel id="jc">
		<frequencies>
			<frequencyModel dataType="nucleotide">
				<frequencies>
					<parameter id="frequencies" value="0.25 0.25 0.25 0.25"/>
				</frequencies>
			</frequencyModel>
		</frequencies>
		<kappa>
			<parameter value="1.0"/>
		</kappa>
	</HKYModel>

	<!-- site model                                                              -->
	<siteModel id="siteModel">
		<substitutionModel>
			<HKYModel idref="jc"/>
		</substitutionModel>
	</siteModel>

	<!--                                                                         -->
	<statistic id="mu" name="mu">
		<siteModel idref="siteModel"/>
	</statistic>
	

	<!-- Likelihood for tree given sequence data                                 -->
	<treeDataLikelihood id="treeLikelihood" useAmbiguities="false">
		<partition>
			<patterns idref="patterns"/>
			<siteModel idref="siteModel"/>
		</partition>
		<treeModel idref="treeModel"/>
		<strictClockBranchRates idref="branchRates"/>
	</treeDataLikelihood>
	

	<!-- Define operators                                                        -->
	<operators id="operators" optimizationSchedule="default">
		<scaleOperator scaleFactor="0.75" scaleAll="true" ignoreBounds="true" weight="3">
			<parameter idref="treeModel.allInternalNodeHeights"/>
		</scaleOperator>
		<subtreeSlide size="1.0" gaussian="true" weight="30">
			<treeModel idref="treeModel"/>
		</subtreeSlide>
		<narrowExchange weight="30">
			<treeModel idref="treeModel"/>
		</narrowExchange>
		<wideExchange weight="3">
			<treeModel idref="treeModel"/>
		</wideExchange>
		<wilsonBalding weight="3">
			<treeModel idref="treeModel"/>
		</wilsonBalding>
		<scaleOperator scaleFactor="0.75" weight="3">
			<parameter idref="treeModel.rootHeight"/>
		</scaleOperator>
		<uniformOperator weight="30">
			<parameter idref="treeModel.internalNodeHeights"/>
		</uniformOperator>
	</operators>
	

	<!-- Define MCMC                                                             -->
	<mcmc id="mcmc" chainLength="100000000" autoOptimize="true" operatorAnalysis="%s.ops">
		<joint id="joint">
			<prior id="prior">
				<coalescentLikelihood idref="coalescent"/>
				
				
				<strictClockBranchRates idref="branchRates"/>
			</prior>
			<likelihood id="likelihood">
				<treeDataLikelihood idref="treeLikelihood"/>
			</likelihood>
		</joint>
		<operators idref="operators"/>

		<!-- write log to screen                                                     -->
		<log id="screenLog" logEvery="10000">
			<column label="Joint" dp="4" width="12">
				<joint idref="joint"/>
			</column>
			<column label="Prior" dp="4" width="12">
				<prior idref="prior"/>
			</column>
			<column label="Likelihood" dp="4" width="12">
				<likelihood idref="likelihood"/>
			</column>
			<column label="rootHeight" sf="6" width="12">
				<parameter idref="treeModel.rootHeight"/>
			</column>
		</log>

		<!-- write log to file                                                       -->
		<log id="fileLog" logEvery="10000" fileName="%s.log" overwrite="false">
			<joint idref="joint"/>
			<prior idref="prior"/>
			<likelihood idref="likelihood"/>
			<parameter idref="treeModel.rootHeight"/>
			<treeLengthStatistic idref="treeLength"/>
			<parameter idref="constant.popSize"/>
			<parameter idref="clock.rate"/>
			<rateStatistic idref="meanRate"/>
			<treeDataLikelihood idref="treeLikelihood"/>
			<strictClockBranchRates idref="branchRates"/>
			<coalescentLikelihood idref="coalescent"/>
			
		</log>

		<!-- write tree log to file                                                  -->
		<logTree id="treeFileLog" logEvery="10000" nexusFormat="true" fileName="%s.trees" sortTranslationTable="true">
			<treeModel idref="treeModel"/>
			<trait name="rate" tag="rate">
				<strictClockBranchRates idref="branchRates"/>
			</trait>
			<joint idref="joint"/>
		</logTree>
	</mcmc>

	<report>
		<property name="timer">
			<mcmc idref="mcmc"/>
		</property>
	</report>
	
</beast>
""" % (ds,ds,ds))

f.close()

In [30]:
# make xml file for BEAST SS analysis for marginal likelihood

filename = "../dat/"+ds+"/"+ds+"_MLL.xml"
f = open(filename, "w")
f.write('''<?xml version="1.0" standalone="yes"?>

<!-- Generated by BEAUTi v1.10.4 Prerelease #bc6cbd9                         -->
<!--       by Alexei J. Drummond, Andrew Rambaut and Marc A. Suchard         -->
<!--       Department of Computer Science, University of Auckland and        -->
<!--       Institute of Evolutionary Biology, University of Edinburgh        -->
<!--       David Geffen School of Medicine, University of California, Los Angeles-->
<!--       http://beast.community/                                           -->
<beast version="1.10.4">
	

	<!-- The list of taxa to be analysed (can also include dates/ages).          -->
	<!-- ntax=%d                                                                 -->
	<taxa id="taxa">
''' % n_tax)

for species in data:
    f.write('\t\t<taxon id="%s"/> \n' % species)

f.write('''\t</taxa>
	
	

	<!-- The sequence alignment (each sequence refers to a taxon above).         -->
	<!-- ntax=%d nchar=%d                                                      -->
	<alignment id="alignment" dataType="nucleotide">
''' % (n_tax,n_char))

for species in data:
    f.write("\t\t<sequence>\n")
    f.write('\t\t\t<taxon idref="%s"/>\n'%species)
    f.write('\t\t\t%s\n'%data[species])
    f.write("\t\t</sequence>\n")
f.write("\t</alignment>\n")

f.write("""

	<!-- The unique patterns from 1 to end                                       -->
	<patterns id="patterns" from="1" strip="false">
		<alignment idref="alignment"/>
	</patterns>
	

	<!-- A prior assumption that the population size has remained constant       -->
	<!-- throughout the time spanned by the genealogy.                           -->
	<constantSize id="constant" units="years">
		<populationSize>
			<parameter id="constant.popSize" value="5.0"/>
		</populationSize>
	</constantSize>
	

	<!-- Generate a random starting tree under the coalescent process            -->
	<coalescentSimulator id="startingTree">
		<taxa idref="taxa"/>
		<constantSize idref="constant"/>
	</coalescentSimulator>
	

	<!-- Generate a tree model                                                   -->
	<treeModel id="treeModel">
		<coalescentTree idref="startingTree"/>
		<rootHeight>
			<parameter id="treeModel.rootHeight"/>
		</rootHeight>
		<nodeHeights internalNodes="true">
			<parameter id="treeModel.internalNodeHeights"/>
		</nodeHeights>
		<nodeHeights internalNodes="true" rootNode="true">
			<parameter id="treeModel.allInternalNodeHeights"/>
		</nodeHeights>
	</treeModel>

	<!-- Statistic for sum of the branch lengths of the tree (tree length)       -->
	<treeLengthStatistic id="treeLength">
		<treeModel idref="treeModel"/>
	</treeLengthStatistic>
	

	<!-- Generate a coalescent likelihood                                        -->
	<coalescentLikelihood id="coalescent">
		<model>
			<constantSize idref="constant"/>
		</model>
		<populationTree>
			<treeModel idref="treeModel"/>
		</populationTree>
	</coalescentLikelihood>
	

	<!-- The strict clock (Uniform rates across branches)                        -->
	<strictClockBranchRates id="branchRates">
		<rate>
			<parameter id="clock.rate" value="1.0"/>
		</rate>
	</strictClockBranchRates>
	
	<rateStatistic id="meanRate" name="meanRate" mode="mean" internal="true" external="true">
		<treeModel idref="treeModel"/>
		<strictClockBranchRates idref="branchRates"/>
	</rateStatistic>
	

	<!-- The JC substitution model (Jukes & Cantor, 1969)                        -->
	<HKYModel id="jc">
		<frequencies>
			<frequencyModel dataType="nucleotide">
				<frequencies>
					<parameter id="frequencies" value="0.25 0.25 0.25 0.25"/>
				</frequencies>
			</frequencyModel>
		</frequencies>
		<kappa>
			<parameter value="1.0"/>
		</kappa>
	</HKYModel>

	<!-- site model                                                              -->
	<siteModel id="siteModel">
		<substitutionModel>
			<HKYModel idref="jc"/>
		</substitutionModel>
	</siteModel>

	<!--                                                                         -->
	<statistic id="mu" name="mu">
		<siteModel idref="siteModel"/>
	</statistic>
	

	<!-- Likelihood for tree given sequence data                                 -->
	<treeDataLikelihood id="treeLikelihood" useAmbiguities="false">
		<partition>
			<patterns idref="patterns"/>
			<siteModel idref="siteModel"/>
		</partition>
		<treeModel idref="treeModel"/>
		<strictClockBranchRates idref="branchRates"/>
	</treeDataLikelihood>
	

	<!-- Define operators                                                        -->
	<operators id="operators" optimizationSchedule="default">
		<scaleOperator scaleFactor="0.75" scaleAll="true" ignoreBounds="true" weight="3">
			<parameter idref="treeModel.allInternalNodeHeights"/>
		</scaleOperator>
		<subtreeSlide size="1.0" gaussian="true" weight="30">
			<treeModel idref="treeModel"/>
		</subtreeSlide>
		<narrowExchange weight="30">
			<treeModel idref="treeModel"/>
		</narrowExchange>
		<wideExchange weight="3">
			<treeModel idref="treeModel"/>
		</wideExchange>
		<wilsonBalding weight="3">
			<treeModel idref="treeModel"/>
		</wilsonBalding>
		<scaleOperator scaleFactor="0.75" weight="3">
			<parameter idref="treeModel.rootHeight"/>
		</scaleOperator>
		<uniformOperator weight="30">
			<parameter idref="treeModel.internalNodeHeights"/>
		</uniformOperator>
	</operators>
	

	<!-- Define MCMC                                                             -->
	<mcmc id="mcmc" chainLength="10000000" autoOptimize="true" operatorAnalysis="%s.ops">
		<joint id="joint">
			<prior id="prior">
				<coalescentLikelihood idref="coalescent"/>
				
				
				<strictClockBranchRates idref="branchRates"/>
			</prior>
			<likelihood id="likelihood">
				<treeDataLikelihood idref="treeLikelihood"/>
			</likelihood>
		</joint>
		<operators idref="operators"/>

		<!-- write log to screen                                                     -->
		<log id="screenLog" logEvery="1000">
			<column label="Joint" dp="4" width="12">
				<joint idref="joint"/>
			</column>
			<column label="Prior" dp="4" width="12">
				<prior idref="prior"/>
			</column>
			<column label="Likelihood" dp="4" width="12">
				<likelihood idref="likelihood"/>
			</column>
			<column label="rootHeight" sf="6" width="12">
				<parameter idref="treeModel.rootHeight"/>
			</column>
		</log>

		<!-- write log to file                                                       -->
		<log id="fileLog" logEvery="1000" fileName="%s.log" overwrite="false">
			<joint idref="joint"/>
			<prior idref="prior"/>
			<likelihood idref="likelihood"/>
			<parameter idref="treeModel.rootHeight"/>
			<treeLengthStatistic idref="treeLength"/>
			<parameter idref="constant.popSize"/>
			<parameter idref="clock.rate"/>
			<rateStatistic idref="meanRate"/>
			<treeDataLikelihood idref="treeLikelihood"/>
			<strictClockBranchRates idref="branchRates"/>
			<coalescentLikelihood idref="coalescent"/>
			
		</log>

		<!-- write tree log to file                                                  -->
		<logTree id="treeFileLog" logEvery="1000" nexusFormat="true" fileName="%s.trees" sortTranslationTable="true">
			<treeModel idref="treeModel"/>
			<trait name="rate" tag="rate">
				<strictClockBranchRates idref="branchRates"/>
			</trait>
			<joint idref="joint"/>
		</logTree>
	</mcmc>

	<!-- START Marginal Likelihood Estimator                                     -->

	<!-- Define marginal likelihood estimator (PS/SS) settings                   -->
	<marginalLikelihoodEstimator chainLength="500000" pathSteps="50" pathScheme="betaquantile" alpha="0.3">
		<samplers>
			<mcmc idref="mcmc"/>
		</samplers>
		<pathLikelihood id="pathLikelihood">
			<source>
				<joint idref="joint"/>
			</source>
			<destination>
				<prior idref="prior"/>
			</destination>
		</pathLikelihood>
		<log id="MLELog" logEvery="1000" fileName="%s.mle.log">
			<pathLikelihood idref="pathLikelihood"/>
		</log>
	</marginalLikelihoodEstimator>

	<!-- Stepping-stone sampling estimator from collected samples                -->
	<steppingStoneSamplingAnalysis fileName="%s.mle.log" resultsFileName="%s.mle.result.log">
		<likelihoodColumn name="pathLikelihood.delta"/>
		<thetaColumn name="pathLikelihood.theta"/>
	</steppingStoneSamplingAnalysis>

	<!-- END Marginal Likelihood Estimator                                       -->

	<report>
		<property name="timer">
			<mcmc idref="mcmc"/>
		</property>
	</report>
	
</beast>
""" % (ds,ds,ds,ds,ds,ds))

f.close()